In [9]:
import pandas as pd
from data_reciever import datasets
from datetime import datetime
from io import StringIO
import json

In [28]:
date_itemquantity_df=pd.read_csv('../ds/clean/date_itemquantity.csv')
menu_ingredient_df=pd.read_json('../ds/menu_ingredient.json')
item_price_df=pd.read_csv('../ds/clean/date_summary.csv')
sales_history_df=pd.read_csv('../ds/sales_history.csv')

In [11]:
print("xxx weekends check xxx")

# Convert Date to datetime and extract year-month
date_itemquantity_df['Date'] = pd.to_datetime(date_itemquantity_df['Date'])
date_itemquantity_df['YearMonth'] = date_itemquantity_df['Date'].dt.to_period('M')

# Count weekends per month (only dates where Weekend = 1)
weekends_per_month = date_itemquantity_df[date_itemquantity_df['Weekend'] == 1].groupby('YearMonth')['Date'].nunique().reset_index()
weekends_per_month.columns = ['YearMonth', 'Number_of_Weekends']

print("Number of weekends in each month:")
print("=" * 35)
for _, row in weekends_per_month.iterrows():
    print(f"{row['YearMonth']}: {row['Number_of_Weekends']} weekends")

# Alternative: Simple display
print("\n" + "=" * 35)
print(weekends_per_month.to_string(index=False))

xxx weekends check xxx
Number of weekends in each month:
2025-01: 8 weekends
2025-02: 8 weekends
2025-03: 10 weekends
2025-04: 8 weekends
2025-05: 9 weekends
2025-06: 9 weekends
2025-07: 8 weekends
2025-08: 10 weekends
2025-09: 8 weekends
2025-10: 8 weekends

YearMonth  Number_of_Weekends
  2025-01                   8
  2025-02                   8
  2025-03                  10
  2025-04                   8
  2025-05                   9
  2025-06                   9
  2025-07                   8
  2025-08                  10
  2025-09                   8
  2025-10                   8


In [27]:
print("xxx daily sales xxx")


df_sales = pd.read_csv("../ds/clean/date_itemquantity.csv")
menu_ingredient_df = pd.read_json("../ds/menu_ingredient.json")
df_menu = menu_ingredient_df[["name", "price"]]
df_merged = pd.merge(df_sales, df_menu, left_on="Dish", right_on="name", how="left")
df_merged["sale_amount"] = df_merged["Quantity"] * df_merged["price"]
daily_sales = df_merged.groupby("Date")["sale_amount"].sum().reset_index()
print(daily_sales)
store_avg_sale_day = daily_sales["sale_amount"].sum() / len(daily_sales)
print("Average daily sale:", store_avg_sale_day)


xxx daily sales xxx
           Date  sale_amount
0    2024-10-01         9090
1    2024-10-02         7760
2    2024-10-03         9060
3    2024-10-04         7270
4    2024-10-05        13180
..          ...          ...
390  2025-10-26        14320
391  2025-10-27        12920
392  2025-10-28         9930
393  2025-10-29         8710
394  2025-10-30        13440

[395 rows x 2 columns]
Average daily sale: 9592.075949367088


In [ ]:
print ("xxx daily sales check xxx")

# --- 8️⃣ Read actual sales history ---
sales_history_df = pd.read_csv("../ds/sales_history.csv", header=None,
                               names=["CustomerID", "Date", "Items_Ordered", "Total_Cost_INR", "Season", "isEvent",""])

# --- 9️⃣ Group by Date and sum total recorded sales ---
daily_recorded = sales_history_df.groupby("Date")["TotalCost"].sum().reset_index()

# --- 🔟 Merge the calculated and recorded daily sales ---
comparison = pd.merge(daily_sales, daily_recorded, on="Date", how="inner",
                      suffixes=("_calc", "_recorded"))

# --- 11️⃣ Calculate difference and accuracy ---
comparison["difference"] = abs(comparison["sale_amount_calc"] - comparison["TotalCost_recorded"])
comparison["accuracy(%)"] = 100 * (1 - (comparison["difference"] / comparison["TotalCost_recorded"]))

print("\n🔹 Daily Comparison between calculated and recorded totals:")
print(comparison)

# --- 12️⃣ Calculate average accuracy across all days ---
avg_accuracy = comparison["accuracy(%)"].mean()
print(f"\n✅ Average matching accuracy: {avg_accuracy:.2f}%")

# --- 13️⃣ Average daily sale from recorded history ---
store_avg_sale_day_history = daily_recorded["TotalCost"].sum() / len(daily_recorded)
print("Average daily sale (from history):", store_avg_sale_day_history)